# LightGBM model

In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import sys

# go up two directories
sys.path.append('../../')
from codecompasslib.API.drive_operations import get_creds_drive, list_shared_drive_contents, download_csv_as_pd_dataframe
from codecompasslib.API.get_bulk_data import get_stared_repos

import openai
client = openai.Client(api_key='YOUR API KEY')

c:\Users\ketis\anaconda3\envs\codecompassvenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token loaded successfully.


In [2]:
DRIVE_ID = "0AL1DtB4TdEWdUk9PVA"
DATA_FOLDER = "13JitBJQLNgMvFwx4QJcvrmDwKOYAShVx"

creds = get_creds_drive()
list_shared_drive_contents(creds=creds, folder_id=DATA_FOLDER, drive_id=DRIVE_ID)


Files in the folder:
test.csv (1hAP9CD6iP4FSZP4RSRm2CYUrS2KF_Lhf)
uploaded_dataset.csv (1WSgwAhzNbSqC6e_RRBDHpgpQCnGZvVcc)
dataset.csv (1AdJGrqauyeOzjyaT0752hTJrkQjbEvfb)
allReposCleaned.csv (1jIYBQQJNo2s1bo3LHlYgKzUNNM0ueuhQ)


True

In [3]:
df = download_csv_as_pd_dataframe(creds,"1WSgwAhzNbSqC6e_RRBDHpgpQCnGZvVcc")
df.shape


Download 11%.

Download 23%.

Download 35%.

Download 47%.

Download 59%.

Download 71%.

Download 83%.

Download 95%.

Download 100%.


c:\Users\ketis\UniversityStuff\2024\RecAndChat\CodeCompass\codecompasslib\models\../..\codecompasslib\API\drive_operations.py:88: DtypeWarning: Columns (6,11,12,15,16,17,18,19,20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  return read_csv(fh)


(2579859, 28)

In [4]:
# retrieve only the columns that are needed id, name, owner_user, description, url, date_created, date_updated, stars, language, topics
df_extracted_columns = df[['id', 'name', 'owner_user', 'description', 'url', 'date_created', 'date_updated', 'stars', 'language', 'topics']]
df_extracted_columns.head()

,id,name,owner_user,description,url,date_created,date_updated,stars,language,topics
0,444741024.0,doc.aurora.dev-develop-compat-evm,mercyog,No description,https://api.github.com/repos/mercyog/doc.auror...,2022-01-05T09:24:59Z,2023-03-05T00:05:37Z,1,NaN,[]
1,404812692.0,White_Paper,mercyog,White Paper for Choice Coin,https://api.github.com/repos/mercyog/White_Paper,2021-09-09T17:30:02Z,2023-03-05T00:05:37Z,1,NaN,[]
2,462013111.0,Advance-SQL-Assignment,Rameshwar0852,Advance SQL assignment.,https://api.github.com/repos/Rameshwar0852/Adv...,2022-02-21T20:01:28Z,2024-03-09T14:13:51Z,1,NaN,[]
3,462013123.0,anosql,Rameshwar0852,Easy SQL in Python,https://api.github.com/repos/Rameshwar0852/anosql,2022-02-21T20:01:29Z,2024-03-09T14:13:51Z,1,NaN,[]
4,585055299.0,Automation_Project,Rameshwar0852,Automated Bash Script to automate log Backup g...,https://api.github.com/repos/Rameshwar0852/Aut...,2023-01-04T07:49:35Z,2024-03-09T14:13:42Z,1,Shell,[]


In [5]:
# check for missing values
df_extracted_columns.isnull().sum()

id                    1
name                 37
owner_user            0
description          39
url                   1
date_created          1
date_updated          1
stars                 1
language        1058582
topics                2
dtype: int64

In [6]:
# remove rows with missing values (We still have a very big dataset after removing the missing values anyway)
df_clean = df_extracted_columns.dropna()

### Getting starred repos

In [7]:
# choosing a target user to generate recommendations for
target_user = 'Rameshwar0852'

In [8]:
starred_by_target = get_stared_repos(target_user)

In [9]:
# retrieve user_owner of starred repos
for repo in starred_by_target[0]:
    print(repo['owner_user'])

Rameshwar0852
Rameshwar0852
Rameshwar0852
Rameshwar0852
Rameshwar0852
Rameshwar0852
Rameshwar0852
Rameshwar0852
Rameshwar0852
Rameshwar0852
Rameshwar0852
Rameshwar0852
Rameshwar0852
Rameshwar0852
Rameshwar0852
Rameshwar0852
Rameshwar0852
Rameshwar0852
Rameshwar0852
Rameshwar0852
Rameshwar0852
Rameshwar0852
Rameshwar0852
Rameshwar0852
bnsreenu
nikhilroxtomar
Rameshwar0852
iam-veeramalla
youssefHosni
Rameshwar0852
Timsal123
amalkumar308
amalkumar308
amalkumar308
amalkumar308
amalkumar308
amalkumar308
amalkumar308
amalkumar308
amalkumar308
sachinOraon


## !! TO-Do write logic in case a target user's starred repo is not in our df

### embedding the dataset

In [10]:
df_clean.head()

,id,name,owner_user,description,url,date_created,date_updated,stars,language,topics
4,585055299.0,Automation_Project,Rameshwar0852,Automated Bash Script to automate log Backup g...,https://api.github.com/repos/Rameshwar0852/Aut...,2023-01-04T07:49:35Z,2024-03-09T14:13:42Z,1,Shell,[]
18,227460783.0,IKON,Rameshwar0852,CBIR(CONTENt BASED IMAGE RETRIVALE APPLICATION...,https://api.github.com/repos/Rameshwar0852/IKON,2019-12-11T21:08:53Z,2019-12-12T09:01:55Z,0,Python,[]
19,666532441.0,javamavensonarrgohelmk8,Rameshwar0852,No description,https://api.github.com/repos/Rameshwar0852/jav...,2023-07-14T19:08:36Z,2024-03-09T14:13:28Z,1,HTML,[]
23,717350370.0,node001,Rameshwar0852,files_repo,https://api.github.com/repos/Rameshwar0852/nod...,2023-11-11T07:42:16Z,2024-03-09T14:13:21Z,1,JavaScript,[]
24,717665372.0,nodeandjs,Rameshwar0852,node java script application,https://api.github.com/repos/Rameshwar0852/nod...,2023-11-12T06:59:07Z,2024-03-09T14:13:19Z,1,JavaScript,[]


In [11]:
def get_embeddings(strings_to_embed):
    response = client.embeddings.create(
        input=strings_to_embed,
        model="text-embedding-3-small"  # You can choose the model you prefer
    )
    return response

In [12]:
# cut description if it's size exceeds 8000 tokens
df_clean['description'] = df_clean['description'].apply(lambda x: x[:8190]) # due to openAI API limit

C:\Users\ketis\AppData\Local\Temp\ipykernel_39424\3525878226.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['description'] = df_clean['description'].apply(lambda x: x[:8190]) # due to openAI API limit


In [13]:
print(len(df_clean['description'].values.tolist()))

1521224


In [14]:
df_clean['description'].values.tolist()

['Automated Bash Script to automate log Backup generated by webserver.ubuntu22.LTS.jammy',
 'CBIR(CONTENt BASED IMAGE RETRIVALE APPLICATION BASED ON HSV',
 'No description',
 'files_repo',
 'node java script application ',
 'versioning integrated with jenkins ',
 'testing_node_app on k8',
 'No description',
 'pv_hawk installations',
 'Códigos sobre PyQt5',
 'reddit_ingress ciserver---cdserver',
 'Project Based on Deep Learning & Python',
 'Computer theory learning notes',
 'A small C11 compiler',
 'A development environment for C++',
 'Crawl novels from sfacg, ciweimao, esjzone, lightnovel and masiro; generate, append and extract epub',
 'A C++ library, which wraps several important C libraries and provides some convenient functions',
 'C++ Library Manager',
 'Programming learning notes',
 'Python learning notes',
 'Rust learning notes',
 'Web learning notes',
 'Curated collection of useful JavaScript snippets that you can understand in 30 seconds or less.',
 'A curated collection of u

In [16]:
# check if there is any description that is empty = ''
df_clean['description'].apply(lambda x: x == '').sum()

0

In [17]:
# extract description as list of strings
descriptions = df_clean['description'].values.tolist()
# extract id
ids = df_clean['id'].values.tolist()

# create an emptry dataframe to store the embeddings
df_with_embeddings = pd.DataFrame(columns=['id', 'description', 'description_embedding'])

# iterate over every 2048 descriptions (2048 is a limit that openai has for the number of strings that can be embedded at once)
for i in range(0, len(descriptions), 2040):
    print(i)
    # get the descriptions in a list
    descriptions_batch = descriptions[i:i+2040]
    # get the embeddings for the descriptions
    embeddings_response = get_embeddings(descriptions_batch)
    
    # Extract the embeddings from the response
    embeddings = [embedding.embedding for embedding in embeddings_response.data]
    rows_to_add = []
    
    # Inside your loop, create a dictionary for the new row
    for j in range(len(embeddings)):
        new_row = {
            'id': ids[i+j],
            'description': descriptions_batch[j],
            'description_embedding': [embeddings[j]]  # Store the embedding as a list
        }
        # Convert the dictionary to a DataFrame and add it to the list
        rows_to_add.append(pd.DataFrame(new_row, index=[0]))

    # After the loop, concatenate all the DataFrames in the list to the original DataFrame
    df_with_embeddings = pd.concat([df_with_embeddings] + rows_to_add, ignore_index=True)

0


C:\Users\ketis\AppData\Local\Temp\ipykernel_39424\3492201114.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_with_embeddings = pd.concat([df_with_embeddings] + rows_to_add, ignore_index=True)


2040
4080
6120
8160
10200
12240
14280
16320
18360
20400
22440
24480
26520
28560
30600
32640
34680
36720
38760
40800
42840
44880
46920
48960
51000


In [ ]:
df_with_embeddings.head()

,id,description,description_embedding
0,585055299.0,Automated Bash Script to automate log Backup g...,"[0.009664751589298248, -0.02779489941895008, 0..."
1,227460783.0,CBIR(CONTENt BASED IMAGE RETRIVALE APPLICATION...,"[-0.010402947664260864, 0.0059961434453725815,..."
2,666532441.0,No description,"[-0.04160582274198532, 0.02379406988620758, -0..."
3,717350370.0,files_repo,"[-0.030517788603901863, -0.02503896690905094, ..."
4,717665372.0,node java script application,"[-0.037780020385980606, 0.0437803752720356, 0...."


In [ ]:
strings_to_embed = df['description'].tolist()[:2000]

# Call the function and get the embeddings
embeddings_response = get_embeddings(strings_to_embed)

# Extract the embeddings from the response
embeddings = [embedding.embedding for embedding in embeddings_response.data]
print(len(embeddings))
embeddings = embeddings[:5]
for i in range(len(embeddings)):
    print(strings_to_embed[i], embeddings[i])

In [54]:
# import pandas as pd
# from langchain_community.embeddings import MistralEmbeddings

# # Initialize Mistral embeddings with the 7B model
# embeddings = MistralEmbeddings(model_name='mistral-7b')

# # Assuming 'df' is your DataFrame and 'text_column' is the name of the textual column
# # Function to convert a column of text to embeddings
# def convert_text_column_to_embeddings(dataframe, text_column):
#     # Create a new column for embeddings
#     dataframe[f'{text_column}_embeddings'] = None
    
#     # Convert each text entry to an embedding and store it in the new column
#     dataframe[f'{text_column}_embeddings'] = dataframe[text_column].apply(lambda x: embeddings.embed_query(x))

# # Call the function to convert the text column to embeddings
# convert_text_column_to_embeddings(df, 'description')

ImportError: cannot import name 'MistralEmbeddings' from 'langchain_community.embeddings' (c:\Users\ketis\anaconda3\envs\codecompassvenv\lib\site-packages\langchain_community\embeddings\__init__.py)

In [69]:
# import requests
# import numpy as np

# res = requests.post('http://localhost:11434/api/embeddings', json={'model' : 'codellama:7b', 'prompt' : 'This is a test query', 'device' : 'gpu'})
# print(res.json())

{'embedding': [0.5749330520629883, 0.33146896958351135, 0.4243919849395752, 0.44006428122520447, -0.7008244395256042, -0.3326490521430969, 0.687189519405365, -1.0797873735427856, 0.2004203498363495, 0.6842340230941772, 0.48517468571662903, -0.4508664309978485, -0.6013938784599304, 0.15179386734962463, -0.009807542897760868, -0.7662606835365295, 0.22647151350975037, 0.6025199294090271, 0.40927162766456604, 0.039661943912506104, 1.011734127998352, -0.778490424156189, 0.705863893032074, 0.3796637952327728, 0.6322203278541565, 0.045994099229574203, 0.2356528639793396, -0.3259495496749878, -0.21417587995529175, 0.3098725974559784, 0.3808402121067047, -0.30459296703338623, 0.4138452708721161, 1.8199622631072998, 0.029673265293240547, -0.100958451628685, -0.7713375091552734, 0.128494530916214, -0.0454881452023983, 0.5043752193450928, 0.43134284019470215, 0.38808029890060425, 1.0358290672302246, -0.04148095101118088, 0.21453525125980377, 0.664347231388092, -0.32186469435691833, -0.314150899648

In [85]:
# # Load a pre-trained Sentence Transformer model
# model_name = 'stsb-roberta-base'
# model = SentenceTransformer(model_name)

## tESTING EMBEDDING

In [86]:
# # Sentence for which you want to generate embedding
# sentence = "How are you doing today. iS EVERYTHING OKAY?"

# # Generate embedding for the sentence
# embedding = model.encode(sentence)

# # The 'embedding' variable now contains the embedding vector for the sentence
# print("Embedding shape:", embedding.shape)
# print("Embedding vector:", embedding[:5])

Embedding shape: (768,)
Embedding vector: [-1.0853684   0.08040027  0.2953426  -0.49769846  0.09283736]


## Embedding the dataframe

In [93]:
# Generate empty dataframe X
X = pd.DataFrame()
embeddings = []

# Take column description and change every row into an embedding with a for loop
for i, row in df.iterrows():
    if i % 100 == 0:
        print(f'Processing row {i}')
    # Generate embedding for the sentence
    embedding = model.encode(row['description'])
    # Append the embedding to the list
    embeddings.append(embedding)
    if i > 10000:
        break

# Convert the list of embeddings into a DataFrame
X = pd.DataFrame(embeddings)


Processing row 100
Processing row 200
Processing row 400
Processing row 500
Processing row 700
Processing row 800
Processing row 900
Processing row 1000
Processing row 1100
Processing row 1300
Processing row 1500
Processing row 1600
Processing row 1800
Processing row 2000
Processing row 2300
Processing row 2500
Processing row 2700
Processing row 3000
Processing row 3300
Processing row 3400
Processing row 3500
Processing row 3600
Processing row 3800
Processing row 3900
Processing row 4200
Processing row 4900
Processing row 5000
Processing row 5100
Processing row 5200
Processing row 5300
Processing row 5400
Processing row 5500
Processing row 5600
Processing row 5700
Processing row 5800
Processing row 6100
Processing row 6300
Processing row 6400
Processing row 6500
Processing row 6700
Processing row 6800
Processing row 7200
Processing row 7300
Processing row 7400
Processing row 7500
Processing row 7600
Processing row 7800
Processing row 8100
Processing row 8400
Processing row 8500
Process

In [95]:
X.shape

(6522, 768)

In [97]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.469418,-0.998858,-0.148037,1.412736,1.090872,-0.367098,-0.021415,-0.515937,0.151782,-0.172840,...,-0.356760,-0.755197,-0.768200,-0.059507,0.269944,0.076307,0.740030,-1.630054,-0.199989,0.720446
1,0.449717,-0.399685,0.586259,-0.155754,-0.322601,-1.048024,-0.384903,-0.187562,0.320055,0.739774,...,-1.233410,-1.025798,-1.398393,-0.052006,0.202888,0.060399,-0.165392,0.695042,0.091617,0.101518
2,0.003796,-1.227154,-0.238535,-1.416075,-0.049791,0.472089,0.466126,0.592889,-1.267169,0.364745,...,-0.096014,-0.063959,-1.136193,-0.598189,-0.238775,0.518613,0.570125,0.001731,0.371637,-0.548268
3,0.312427,-0.572282,0.164659,-0.739483,-0.792593,-0.320942,0.096133,-0.656378,0.749682,0.211361,...,-0.426190,-0.757543,0.330005,-0.323000,0.910003,0.238358,0.073861,-0.123928,0.017333,-0.040631
4,-0.093899,-1.065263,-0.397888,-0.539837,-0.708843,0.979032,-0.057882,-0.173538,0.018974,-0.313976,...,-0.086232,-0.051804,-0.619846,-0.001891,0.543076,0.391196,-0.451041,-0.267549,-0.598728,0.135250


In [96]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2")
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")

inputs = tokenizer('Can you help me write a formal email to a potential business partner proposing a joint venture?', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=30)
text = tokenizer.batch_decode(outputs)[0]
print(text)

OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like microsoft/phi-2 is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [49]:
# Prepare the features. 'description', 'language'.
X = df[['description', 'language']]

# Create target column which is 1 for the repos that user has starred (for now it is hardcoded to be 1 if the repo belongs to the user, we will change it later on)
df['target'] = df['owner_user'].apply(lambda x: 1 if x == target_user else 0)

# 'target' is whether the user liked the repo or not
y = df['target']

# Embedding X
X['description_embedding'] = X['description'].apply(lambda x: model.encode(x))
X['language_embedding'] = X['language'].apply(lambda x: model.encode(x))

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a LightGBM data matrix from the training set
lgb_train = lgb.Dataset(X_train, y_train)

# Specify the configuration of the model
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

# Train the model
gbm = lgb.train(params, lgb_train, num_boost_round=20)

# Predict the likelihood of 'liking' for the test set
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: description: object, language: object

### Embeddings

In [65]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model='ollama-7b', device='gpu')

text = "This is a test document."

query_result = embeddings.embed_query(text)
query_result[:5]

ValidationError: 1 validation error for OllamaEmbeddings
device
  extra fields not permitted (type=value_error.extra)

In [24]:
# Example text
text = "This is a test document."

In [25]:
query_result = embeddings.embed_query(text)

ValueError: Error raised by inference endpoint: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/embeddings (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001D05A8D0DC0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [18]:
df = pd.read_csv('../Data/original/dataset.csv', usecols=['owner_user', 'name', 'description', 'language'])
df.head(3)

,name,owner_user,description,language
0,doc.aurora.dev-develop-compat-evm,mercyog,No description,NaN
1,White_Paper,mercyog,White Paper for Choice Coin,NaN
2,Advance-SQL-Assignment,Rameshwar0852,Advance SQL assignment.,NaN
